In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import preprocess_dataset as ut
from scipy.interpolate import BSpline
from tfkan import DenseKAN


In [5]:
x_train = tf.convert_to_tensor(pd.read_csv("datasets/x_train.csv").to_numpy()[:, 1:])
y_train = tf.convert_to_tensor(pd.read_csv("datasets/y_train.csv").to_numpy()[:, 1:])

x_val = tf.convert_to_tensor(pd.read_csv("datasets/x_val.csv").to_numpy()[:, 1:])
y_val = tf.convert_to_tensor(pd.read_csv("datasets/y_val.csv").to_numpy()[:, 1:])

x_test = tf.convert_to_tensor(pd.read_csv("datasets/x_test.csv").to_numpy()[:, 1:])
y_test = tf.convert_to_tensor(pd.read_csv("datasets/y_test.csv").to_numpy()[:, 1:])

print(x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_test.shape, y_val.shape)

(70000, 26) (15000, 26) (15000, 26) (70000, 1) (15000, 1) (15000, 1)


In [6]:
kan_layers = [4,1]
kan_model = ut.build_model(kan_layers, x_train.shape[1], mlp=False)
kan_model.summary()

mlp_layers = [20,3,1]
mlp_model = ut.build_model(mlp_layers, x_train.shape[1], mlp=True)
mlp_model.summary()
kan_history = kan_model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_val, y_val), verbose=1)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_kan (DenseKAN)            │ (None, 4)              │           940 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_kan_1 (DenseKAN)          │ (None, 1)              │            37 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 977 (7.63 KB)

 Trainable params: 977 (7.63 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 20)             │           540 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            63 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 607 (2.37 KB)

 Trainable params: 607 (2.37 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 210.9825 - mse: 210.9825 - val_loss: 7.7773 - val_mse: 7.7773
Epoch 2/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 5.9871 - mse: 5.9871 - val_loss: 2.1247 - val_mse: 2.1247
Epoch 3/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 1.7665 - mse: 1.7665 - val_loss: 1.4284 - val_mse: 1.4284
Epoch 4/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 1.3973 - mse: 1.3973 - val_loss: 1.3781 - val_mse: 1.3781
Epoch 5/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.3675 - mse: 1.3675 - val_loss: 1.3617 - val_mse: 1.3617
Epoch 6/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.3391 - mse: 1.3391 - val_loss: 1.3473 - val_mse: 1.3473
Epoch 7/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.3140 - mse: 1.3140 - val_loss: 1.3227 - val_mse: 1.3227
Epoch 8/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.3167 - mse: 1.3167 - val_loss: 1.3215 - val_mse: 1.3215
Epoch 9/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step 

In [11]:
# MLP Pura

pure_mlp_layers = [64, 32, 16, 1]
mlp_model = ut.build_model(pure_mlp_layers, x_train.shape[1], mlp=True, optimizer='rmsprop', loss="categorical_crossentropy")
mlp_model.summary()

mlp_history = mlp_model.fit(x_train, y_train, epochs=50, batch_size=128, validation_data=(x_test, y_test), verbose=1)

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_19 (Dense)                │ (None, 64)             │         1,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,353 (17.00 KB)

 Trainable params: 4,353 (17.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 4.7835e-07 - mse: 40.4156 - val_loss: 4.7640e-07 - val_mse: 40.5267
Epoch 2/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 4.7804e-07 - mse: 40.2649 - val_loss: 4.7640e-07 - val_mse: 40.5267
Epoch 3/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 4.7476e-07 - mse: 40.4069 - val_loss: 4.7640e-07 - val_mse: 40.5267
Epoch 4/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 4.7626e-07 - mse: 40.0418 - val_loss: 4.7640e-07 - val_mse: 40.5267
Epoch 5/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 4.7773e-07 - mse: 40.3373 - val_loss: 4.7640e-07 - val_mse: 40.5267
Epoch 6/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.7521e-07 - mse: 40.1416 - val_loss: 4.7640e-07 - val_mse: 40.5267
Epoch 7/50
234/547 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step - loss: 4.7539e-07 - mse: 39.8079